In [37]:
import sys
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical


In [5]:
# load ascii text and covert to lowercase
filename = "alice.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [6]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [10]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163779
Total Vocab:  58


In [11]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163679


In [22]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

In [30]:
# define the LSTM model
model = Sequential()
model.add(LSTM(2, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [32]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
tensorboard_callback = tensorflow.keras.callbacks.TensorBoard()
callbacks_list = [checkpoint, tensorboard_callback]
# fit the model
model.fit(X, y, epochs=2, batch_size=64, callbacks=callbacks_list)

Epoch 1/2
163648/163679 [============================>.] - ETA: 0s - loss: 3.2198
Epoch 00001: loss improved from inf to 3.21976, saving model to weights-improvement-01-3.2198-bigger.hdf5
163679/163679 [==============================] - 653s 4ms/step - loss: 3.2198
Epoch 2/2
163648/163679 [============================>.] - ETA: 0s - loss: 3.0644
Epoch 00002: loss improved from 3.21976 to 3.06434, saving model to weights-improvement-02-3.0643-bigger.hdf5
163679/163679 [==============================] - 615s 4ms/step - loss: 3.0643


In [31]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 2)                 32        
_________________________________________________________________
dropout_2 (Dropout)          (None, 2)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 58)                174       
Total params: 206
Trainable params: 206
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
# load the network weights
filename = "weights-improvement-02-3.0643-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" and any additional
terms imposed by the copyright holder.  additional terms will be linked
to the pr "
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [43]:
tensorflow.keras.__version__

'2.1.6-tf'